In [1]:
import numpy as np
from astropy.time import Time, TimeDelta
from astropy.coordinates import get_body_barycentric
import astropy.units as u

# -------------------------------
# CONSTANTS
# -------------------------------
R_earth = 6378.14       # km, Earth's radius
z_parking = 200          # km, parking orbit altitude above Earth
r_parking = R_earth + z_parking  # km, parking orbit radius
mu_earth = 398600.4418   # km^3/s^2, Earth's gravitational parameter

R_moon = 1737.0          # km, Moon radius
mu_moon = 4905           # km^3/s^2, Moon's gravitational parameter
h_capture = 2000.0       # km, lunar capture orbit altitude
r_capture = R_moon + h_capture  # km

# -------------------------------
# EPOCHS
# -------------------------------
epoch_departure = Time('2026-02-05 00:00:00', scale='utc')

# -------------------------------
# FUNCTION TO CONVERT BARYCENTRIC TO KM ARRAY
# -------------------------------
def bary_to_km(vec):
    return np.array([vec.x.to(u.km).value,
                     vec.y.to(u.km).value,
                     vec.z.to(u.km).value])

# -------------------------------
# EARTH-MOON POSITIONS
# -------------------------------
earth_dep = bary_to_km(get_body_barycentric('earth', epoch_departure))
moon_dep  = bary_to_km(get_body_barycentric('moon', epoch_departure))

# Earth-Moon distance at departure
R_moon_orbit = np.linalg.norm(moon_dep - earth_dep)  # km

# -------------------------------
# HOHMANN TRANSFER TO MOON
# -------------------------------
a_T = (r_parking + R_moon_orbit) / 2  # km, semi-major axis
E = -mu_earth / (2 * a_T)             # km^2/s^2, specific orbital energy

v_dep = np.sqrt(2 * (mu_earth / r_parking + E))  # km/s
v_arr = np.sqrt(2 * (mu_earth / R_moon_orbit + E))  # km/s

v_moon_orbit = np.sqrt(mu_earth / R_moon_orbit)  # km/s
v_inf = v_arr - v_moon_orbit                     # km/s, hyperbolic excess at Moon

delta_v_dep = v_dep - np.sqrt(mu_earth / r_parking)  # km/s

# Transfer time
T_transfer = np.pi * np.sqrt(a_T**3 / mu_earth)  # seconds
T_transfer_days = T_transfer / 86400

# Arrival epoch
epoch_arrival = epoch_departure + TimeDelta(T_transfer, format='sec')

# Moon position at arrival
moon_arr = bary_to_km(get_body_barycentric('moon', epoch_arrival))
earth_arr = bary_to_km(get_body_barycentric('earth', epoch_arrival))

# Earth-Moon distance at arrival
distance_arr_km = np.linalg.norm(moon_arr - earth_arr)

# -------------------------------
# LUNAR CAPTURE ORBIT
# -------------------------------
v_target = np.sqrt(mu_moon / r_capture)                 # Circular orbital speed
v_p_hyp  = np.sqrt(v_inf**2 + 2 * mu_moon / r_capture) # Hyperbolic perigee speed
delta_v_capture = v_p_hyp - v_target                    # km/s

T_capture = 2 * np.pi * np.sqrt(r_capture**3 / mu_moon)  # seconds
T_capture_days = T_capture / 86400

# -------------------------------
# OUTPUT RESULTS
# -------------------------------
print("=== Hohmann Transfer ===")
print(f"Semi-major axis of transfer orbit: {a_T:.2f} km")
print(f"Specific orbital energy: {E:.6f} km^2/s^2")
print(f"Departure velocity (v_dep): {v_dep:.4f} km/s")
print(f"Arrival velocity at Moon distance (v_arr): {v_arr:.4f} km/s")
print(f"Hyperbolic excess velocity (v_inf): {v_inf:.4f} km/s")
print(f"Delta-v for departure: {delta_v_dep:.4f} km/s")
print(f"Transfer time: {T_transfer:.2f} s ({T_transfer_days:.2f} days)")
print(f"Moon-Earth distance at departure: {R_moon_orbit:.2f} km")
print(f"Moon-Earth distance at arrival : {distance_arr_km:.2f} km")
print(f"Arrival epoch (UTC): {epoch_arrival.iso}")

print("\n=== Lunar Capture ===")
print(f"Circular orbit velocity at capture: {v_target:.4f} km/s")
print(f"Hyperbolic perigee speed: {v_p_hyp:.4f} km/s")
print(f"Delta-v required for lunar insertion: {delta_v_capture:.4f} km/s")
print(f"Capture orbit period: {T_capture:.2f} s ({T_capture_days:.2f} days)")


=== Hohmann Transfer ===
Semi-major axis of transfer orbit: 196298.85 km
Specific orbital energy: -1.015290 km^2/s^2
Departure velocity (v_dep): 10.9160 km/s
Arrival velocity at Moon distance (v_arr): 0.1860 km/s
Hyperbolic excess velocity (v_inf): -0.8301 km/s
Delta-v for departure: 3.1317 km/s
Transfer time: 432770.45 s (5.01 days)
Moon-Earth distance at departure: 386019.55 km
Moon-Earth distance at arrival : 404260.23 km
Arrival epoch (UTC): 2026-02-10 00:12:50.453

=== Lunar Capture ===
Circular orbit velocity at capture: 1.1457 km/s
Hyperbolic perigee speed: 1.8205 km/s
Delta-v required for lunar insertion: 0.6748 km/s
Capture orbit period: 20494.86 s (0.24 days)


| Parameter                                 | Classical Hohmann | Ephemeris-based (Astropy) |
|-------------------------------------------|-----------------|---------------------------|
| Semi-major axis of transfer orbit (km)    | 195488.57       | 196298.85                 |
| Specific orbital energy (km²/s²)          | -1.019498       | -1.015290                 |
| Departure velocity (km/s)                 | 10.9156         | 10.9160                   |
| Arrival velocity at Moon distance (km/s)  | 0.1868          | 0.1860                    |
| Hyperbolic excess velocity (km/s)         | 0.8315          | -0.8301                   |
| Delta-v for departure (km/s)             | 3.1317          | 3.1317                    |
| Transfer time (s)                         | 430093.66       | 432770.45                 |
| Transfer time (days)                      | 4.98            | 5.01                      |
| Moon-Earth distance at departure (km)     | N/A             | 386019.55                 |
| Moon-Earth distance at arrival (km)       | N/A             | 404260.23                 |
| Circular orbit velocity at lunar capture (km/s) | 1.1457   | 1.1457                    |
| Hyperbolic perigee speed at capture (km/s)| 1.8213          | 1.8205                    |
| Delta-v for lunar insertion (km/s)       | 0.6756          | 0.6748                    |
| Capture orbit period (s)                  | 20494.86        | 20494.86                  |
| Capture orbit period (days)               | 0.24            | 0.24                      |
| Arrival epoch (UTC)                       | N/A             | 2026-02-10 00:12:50.453   |


### Clarification of Differences Between Classical Hohmann and Ephemeris-Based Calculations

1. **Hyperbolic Excess Velocity (\($v_\infty$\))**  
   - In classical Hohmann calculations, the Moon is assumed to be at a fixed distance (384,399 km) and stationary relative to Earth. The resulting \($v_\infty$\) is positive and represents the speed of the spacecraft relative to a circular orbit at the Moon’s distance.  
   - In ephemeris-based calculations using Astropy, the Moon’s actual motion is accounted for. The spacecraft may approach the Moon from behind or ahead in its orbit. This can produce a negative \($v_\infty$\) depending on the chosen reference direction. Physically, only the **magnitude** of \($v_\infty$\) matters for planning the lunar insertion burn.

2. **Departure Δv (\($\Delta v_\text{dep}$\))**  
   - The departure Δv is nearly identical in both methods because it depends primarily on the parking orbit around Earth, which is low (200 km altitude). The slight difference in transfer orbit semi-major axis has negligible effect on the departure burn.

3. **Semi-Major Axis of Transfer Orbit (\($a_T$\))**  
   - In classical Hohmann transfer, \(a_T\) is computed using a fixed Moon distance.  
   - In ephemeris-based transfer, the semi-major axis reflects the **actual Earth–Moon distances** at departure and arrival. Since the Moon moves during the transfer time, \($a_T$\) is slightly larger than the classical value.

4. **Transfer Time**  
   - Classical Hohmann assumes a fixed circular Moon orbit, so the transfer time is determined solely by the nominal semi-major axis.  
   - Ephemeris-based calculations account for the Moon’s motion, which slightly changes the effective distance traveled and thus the transfer duration.

5. **Moon–Earth Distance**  
   - Classical Hohmann assumes a constant 384,399 km.  
   - Ephemeris-based values reflect the actual positions of the Moon and Earth at departure and arrival (386,019 km → 404,260 km), making the calculation more accurate for mission planning.


---